<a href="https://colab.research.google.com/github/Cimbir/Store-Sales-Forecasting/blob/main/model_sarima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install kaggle wandb onnx -Uq
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 35.8 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
!mkdir ~/.kaggle
!echo "{\"username\":\"dachisuramelashvili\",\"key\":\"4202ec60e20b612a9947450bb8aeebb5\"}" > ~/.kaggle
!cp /content/drive/MyDrive/ML/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

/bin/bash: line 1: /root/.kaggle: Is a directory


In [ ]:
!kaggle datasets download -d hamzamohiuddin/isbi-2012-challenge
!unzip isbi-2012-challenge.zip -d ISBI_dataset

Dataset URL: https://www.kaggle.com/datasets/hamzamohiuddin/isbi-2012-challenge
License(s): unknown
  0% 0.00/12.6M [00:00<?, ?B/s]
100% 12.6M/12.6M [00:00<00:00, 845MB/s]
Archive:  isbi-2012-challenge.zip
  inflating: ISBI_dataset/unmodified-data/test/imgs/frame_0001.png  
  inflating: ISBI_dataset/unmodified-data/test/imgs/frame_0002.png  
  inflating: ISBI_dataset/unmodified-data/test/imgs/frame_0003.png  
  inflating: ISBI_dataset/unmodified-data/test/imgs/frame_0004.png  
  inflating: ISBI_dataset/unmodified-data/test/imgs/frame_0005.png  
  inflating: ISBI_dataset/unmodified-data/test/imgs/frame_0006.png  
  inflating: ISBI_dataset/unmodified-data/test/imgs/frame_0007.png  
  inflating: ISBI_dataset/unmodified-data/test/imgs/frame_0008.png  
  inflating: ISBI_dataset/unmodified-data/test/imgs/frame_0009.png  
  inflating: ISBI_dataset/unmodified-data/test/imgs/frame_0010.png  
  inflating: ISBI_dataset/unmodified-data/test/imgs/frame_0011.png  
  inflating: ISBI_dataset/unmodifie

In [ ]:
import torch # Main PyTorch Library
from torch import nn # Used for creating the layers and loss function
from torch.optim import Adam # Adam Optimizer
import torchvision.transforms as transforms # Transform function used to modify and preprocess all the images
from torch.utils.data import Dataset, DataLoader # Dataset class and DataLoader for creating the objects
from sklearn.preprocessing import LabelEncoder # Label Encoder to encode the classes from strings to numbers
import matplotlib.pyplot as plt # Used for visualizing the images and plotting the training progress
from PIL import Image # Used to read the images from the directory
import pandas as pd # Used to read/create dataframes (csv) and process tabular data
import numpy as np # preprocessing and numerical/mathematical operations
import os # Used to read the images path from the directory
import random
import wandb
from sklearn.metrics import classification_report, confusion_matrix

device = "cuda" if torch.cuda.is_available() else "cpu" # detect the GPU if any, if not use CPU, change cuda to mps if you have a mac

print("Device available: ", device)

Device available:  cuda


# Get Data

In [ ]:
image_paths = []
label_paths = []

for images in os.listdir(f'/content/ISBI_dataset/unmodified-data/train/imgs'):
  image_paths.append(f'/content/ISBI_dataset/unmodified-data/train/imgs/{images}')
for labels in os.listdir(f'/content/ISBI_dataset/unmodified-data/train/labels'):
  label_paths.append(f'/content/ISBI_dataset/unmodified-data/train/labels/{labels}')

image_paths.sort()
label_paths.sort()

In [ ]:
data_df = pd.DataFrame(zip(image_paths, label_paths), columns = ['image_paths', 'label_paths'])
data_df.head()

,image_paths,label_paths
0,/content/ISBI_dataset/unmodified-data/train/im...,/content/ISBI_dataset/unmodified-data/train/la...
1,/content/ISBI_dataset/unmodified-data/train/im...,/content/ISBI_dataset/unmodified-data/train/la...
2,/content/ISBI_dataset/unmodified-data/train/im...,/content/ISBI_dataset/unmodified-data/train/la...
3,/content/ISBI_dataset/unmodified-data/train/im...,/content/ISBI_dataset/unmodified-data/train/la...
4,/content/ISBI_dataset/unmodified-data/train/im...,/content/ISBI_dataset/unmodified-data/train/la...


In [ ]:
train=data_df.sample(frac=0.7,random_state=7) # Create training of 70% of the data
test=data_df.drop(train.index) # Create testing by removing the 70% of the train data which will result in 30%

val=test.sample(frac=0.5,random_state=7) # Create validation of 50% of the testing data
test=test.drop(val.index) # Create testing by removing the 50% of the validation data which will result in 50%

In [ ]:
image_transform = transforms.Compose([
    transforms.Resize((572, 572)), # One size for all images
    transforms.ToTensor(), # Convert images to PyTorch tensors
    ]) # Transform all images into one clear format (preprocess all images to same properties)

label_transform = transforms.Compose([
    transforms.Resize((388, 388), interpolation=transforms.InterpolationMode.NEAREST), # One size for all images
    transforms.ToTensor(), # Convert images to PyTorch tensors
    ])

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, dataframe, image_transform=None, label_transform=None):
        self.dataframe = dataframe
        self.image_transform = image_transform
        self.label_transform = label_transform

    def __len__(self):
        return self.dataframe.shape[0]

    def __getitem__(self, idx):
        image_path = self.dataframe.iloc[idx, 0]
        label_path = self.dataframe.iloc[idx, 1]

        image = Image.open(image_path).convert('L')
        label = Image.open(label_path).convert('L')

        if self.image_transform:
            image = self.image_transform(image).to(device)
        if self.label_transform:
            label = self.label_transform(label).to(device)

        return image, label

In [ ]:
LR = 1e-4
BATCH_SIZE = 1
EPOCHS = 1

In [ ]:
train_dataset = CustomImageDataset(train, image_transform, label_transform)
val_dataset = CustomImageDataset(val, image_transform, label_transform)
test_dataset = CustomImageDataset(test, image_transform, label_transform)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Model

In [ ]:
class Net(nn.Module):
    def __init__(self):
      super().__init__()

      self.conv1 = nn.Conv2d(1, 1, kernel_size = 3, padding = 0)
      self.down = nn.MaxPool2d(kernel_size = 2, stride = 2)
      self.conv2 = nn.Conv2d(1, 1, kernel_size = 3, padding = 0)
      self.up = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2)
      self.conv3 = nn.Conv2d(1, 1, kernel_size = 179, padding = 0)

    def forward(self, x):
      x = self.conv1(x)
      x = self.down(x)
      x = self.conv2(x)
      x = self.up(x)
      x = self.conv3(x)
      return x

model = Net().to(device) # Create an instance of the model and move it to the GPU Device


In [ ]:
from torchsummary import summary
summary(model, input_size = (1, 572, 572))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 1, 570, 570]              10
         MaxPool2d-2          [-1, 1, 285, 285]               0
            Conv2d-3          [-1, 1, 283, 283]              10
   ConvTranspose2d-4          [-1, 1, 566, 566]               5
            Conv2d-5          [-1, 1, 388, 388]          32,042
Total params: 32,067
Trainable params: 32,067
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 1.25
Forward/backward pass size (MB): 7.30
Params size (MB): 0.12
Estimated Total Size (MB): 8.67
----------------------------------------------------------------


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss() # Cross Entropy Loss
optimizer = Adam(model.parameters(), lr=LR) # Adam optimizer

In [ ]:
import time

total_loss_train_plot = []
total_loss_validation_plot = []
total_acc_train_plot = []
total_acc_validation_plot = []


for epoch in range(EPOCHS):
  total_acc_train = 0
  total_loss_train = 0
  total_loss_val = 0
  total_acc_val = 0

  for inputs, labels in train_loader:
    optimizer.zero_grad()
    start_time = time.time()
    outputs = model(inputs)
    # Squeeze the channel dimension and convert to long data type
    # outputs: [batch_size, num_classes, 388, 388]
    # labels: [batch_size, 1, 388, 388] -> after squeeze(1) -> [batch_size, 388, 388]
    # This now matches the spatial dimensions for CrossEntropyLoss
    train_loss = criterion(outputs, labels)
    total_loss_train += train_loss.item()
    train_loss.backward()

    # Adjust accuracy calculation as labels are now 3D
    # outputs is (batch_size, num_classes, height, width)
    # labels is (batch_size, height, width)
    # torch.argmax(outputs, axis=1) will give (batch_size, height, width) with class indices
    train_acc = (torch.argmax(outputs, axis = 1) == labels.squeeze(1)).sum().item()
    total_acc_train += train_acc
    optimizer.step()
    end_time = time.time()
    print(f'time : {end_time - start_time}s')

  with torch.no_grad():
    for inputs, labels in val_loader:
      outputs = model(inputs)
      # Squeeze the channel dimension and convert to long data type for validation loss
      # outputs: [batch_size, num_classes, 388, 388]
      # labels: [batch_size, 1, 388, 388] -> after squeeze(1) -> [batch_size, 388, 388]
      val_loss = criterion(outputs, labels.squeeze(1).long())
      total_loss_val += val_loss.item()

      # Adjust accuracy calculation for validation
      val_acc = (torch.argmax(outputs, axis = 1) == labels.squeeze(1)).sum().item()
      total_acc_val += val_acc

  # The total number of pixels in the training set is batch_size * height * width
  # Since labels are resized to 388x388, this is train_dataset.__len__() * 388 * 388
  total_pixels_train = train_dataset.__len__() * 388 * 388
  total_pixels_val = val_dataset.__len__() * 388 * 388

  total_loss_train_plot.append(round(total_loss_train/1000, 4))
  total_loss_validation_plot.append(round(total_loss_val/1000, 4))
  # Calculate accuracy per pixel
  total_acc_train_plot.append(round(total_acc_train/total_pixels_train*100, 4))
  total_acc_validation_plot.append(round(total_acc_val/total_pixels_val*100, 4))

  print(f'''Epoch {epoch+1}/{EPOCHS}, Train Loss: {round(total_loss_train/len(train_loader), 4)} Train Pixel Accuracy {round((total_acc_train)/total_pixels_train * 100, 4)}
              Validation Loss: {round(total_loss_val/len(val_loader), 4)} Validation Pixel Accuracy: {round((total_acc_val)/total_pixels_val * 100, 4)}''')
  print("="*25)

time : 0.938758134841919s
time : 0.3298168182373047s
time : 0.3324925899505615s
time : 0.33393073081970215s
time : 0.33394479751586914s
time : 0.3335909843444824s
time : 0.332045316696167s
time : 0.3321208953857422s
time : 0.3313100337982178s
time : 0.33972764015197754s
time : 0.33327412605285645s
time : 0.3347177505493164s
time : 0.3333723545074463s
time : 0.3347949981689453s
time : 0.33531904220581055s
time : 0.33675360679626465s
time : 0.33385491371154785s
time : 0.33472394943237305s
time : 0.3344895839691162s
time : 0.3354015350341797s
time : 0.33641982078552246s


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
